In [16]:
import pickle
len_seq=5
with open('/root/autodl-nas/workspace/datasets/twitch/100k_c.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
pop_user=[]
for u in user_dict.keys():
    if len(user_dict[u])<len_seq+3:
        pop_user.append(u)
for i in pop_user:
    user_dict.pop(i)

In [17]:
items_num=max(max(user_dict.values()))
maxlen=30

batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=3
num_heads=1
interest_num=2
kernel_size=4
transformerencoder=0
multi_dense=0
neg_num=4
test_neg_num=1000

In [18]:
print(len(user_dict))

74617


In [19]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        original_hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(original_hist)<len_seq+3):
            original_hist.insert(0,0)
        #print(len(hist),hist)
        hist=original_hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=original_hist[-len_seq-2:-2]
        nxt = original_hist[-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=original_hist[-len_seq-1:-1]
        nxt = original_hist[-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [20]:
train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)

1075499 74617 74617 74617


In [21]:
users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}

users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}

users, click_seqs, pos_items,neg_items = zip(*val_set)
val_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}

In [22]:
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f, -1)
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f, -1)
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f, -1)

In [23]:
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/twitch/twitch_val_data.pkl', 'rb') as f:
    val_data=pickle.load(f)

In [24]:
print(np.shape(train_data['click_seq']),np.shape(test_data['neg_item']),np.shape(val_data['neg_item']))

(1075499, 5) (74617, 1000) (74617, 4)


In [33]:
for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(test_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(test_data['click_seq'][0],test_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(val_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(val_data['click_seq'][0],val_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
print('pass')

0
500000
1000000
0
500000
1000000
pass


In [34]:
import pickle
with open('/root/autodl-nas/workspace/datasets/twitch/100k_c.pkl', 'rb') as f:    
    user_dict = pickle.load(f)

In [50]:
inte_lenth=[]
for i in user_dict.keys():
    inte_lenth.append(len(user_dict[i]))

In [51]:
import numpy as np
np.mean(inte_lenth)

32.28107995942082

In [52]:

np.percentile(inte_lenth, (25, 50, 75), interpolation='midpoint')

array([10., 19., 42.])

In [7]:
train_data['click_seq'][0:100]

array([[16899,  6081,   690,  2824,  2465],
       [27431, 12413,  2409, 11809, 10835],
       [ 2733,  3710,  2946,  2946, 20576],
       [ 3758, 38168,  1694,  3758, 38168],
       [ 9632,  2345,   191, 24833,  5697],
       [   43,  2425,    43,  2561,     9],
       [   49,  3455,   187,   898,   187],
       [  274,    99,    99,    48,  7138],
       [ 4457,  3250,   293,  1295,   301],
       [  395,  2255,   371,   367,   715],
       [ 1693,   107,  1154,  1132,   305],
       [   54,    54,   468,   468,   573],
       [   54,   632,    99,   183,   710],
       [  233,    16,   233,    16,    16],
       [  314,   305,  1840,  1619,   314],
       [  193,   201,   696,  5809,  2169],
       [  143, 30444,  3003,   133,  1790],
       [  526,    50,   100,    50,   100],
       [ 1584, 10397,  2401,  2401,  1065],
       [ 6954, 11311, 29610,  9999, 33631],
       [  658,   269,   263,   265,   265],
       [ 3950, 16148,  2881, 10137, 16148],
       [ 3673,  9557,  3674,  95

In [10]:
test_data['neg_item'][1]

array([29327, 31321, 39634, 19427, 27842, 29245, 14063, 22930,  5291,
         753,  9740, 19366, 27359, 37451, 18612, 11028, 31973, 23062,
       18702, 11067,  1561,  9303,  3992, 16071, 28932,  7286,  9267,
       27365,  4478, 25844, 10509, 31770, 18680,  3792, 16586, 10335,
       27264,  2248, 31893, 28644, 25590, 16950, 40710,  2836, 38193,
       14787, 39916, 13253, 41215, 23710, 12310, 27812,  1829, 13697,
        8109, 25846, 43108, 27653, 36661, 17129, 11990,  7682, 31439,
       12099, 37552, 38692, 23979, 16651, 17621,  4880, 37641, 22882,
       24655, 27989,  1127, 29921, 31246, 40672, 15304, 31104, 25868,
       28267, 35669, 39220, 24032,  2724, 26351, 36428, 31334,  5352,
       25268, 37939, 34224,  7965, 27974, 20753, 13832, 25487, 10664,
        8953])